---
**llama-index with chroma-db** 

---

### import dependencies and env variables

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# to load open ai key
from dotenv import load_dotenv
import os

### env variables and constants

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

DATA_PATH = '../data'
VECTOR_STORE_PATH = '../chroma_db'

CHUNK_SIZE=512
CHUNK_OVERLAP=10

Python-dotenv could not parse statement starting at line 2


### Logging

In [3]:
#import logging
#import sys
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### RAG pipeline

#### Loading

In [4]:
documents = SimpleDirectoryReader(DATA_PATH).load_data()

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)


In [5]:
len(documents)

827

##### create client and a new collection for chroma db

In [6]:
db = chromadb.PersistentClient(path=VECTOR_STORE_PATH)
chroma_collection = db.get_or_create_collection("quickstart")

#### set up ChromaVectorStore

In [7]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#### indexing and saving to disk

In [8]:
text_splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

Settings.text_splitter = text_splitter

# after we pass storage_context, chroma automatically saves data to disk
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
    show_progress=True
) 

/home/cuphead/Projects/llama-index/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 2025/2025 [00:53<00:00, 37.51it/s]


#### load from disk

In [9]:
db2 = chromadb.PersistentClient(path=VECTOR_STORE_PATH)
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
loaded_index = VectorStoreIndex.from_vector_store(
    vector_store,
)

**Quels sont les critères pour la reconnaissance d'un élément de propriété, d'équipement et de matériel (PPE) comme actif selon les politiques comptables IFRS décrites ?**

**Réponse:**
Un élément de propriété, d'équipement et de matériel (PPE) est reconnu comme actif si, et seulement si, il est probable que les avantages économiques futurs associés à l'élément afflueront vers le Groupe et que son coût peut être mesuré de manière fiable.

**Comment le groupe comptabilise-t-il les dépenses engagées après qu'un actif PPE est prêt à l'emploi, comme les réparations et l'entretien ?**

**Réponse:**
Les dépenses engagées après qu'un actif PPE est prêt à l'emploi, telles que les réparations et l'entretien, sont comptabilisées en charges dans le compte de résultat pour la période au cours de laquelle ces coûts sont engagés. Toutefois, si les dépenses peuvent être mesurées de manière fiable et qu'il est probable que des avantages économiques futurs en découlent pour le Groupe, elles sont incluses dans la valeur comptable de l'actif ou reconnues comme un actif distinct, selon le cas.

#### Query Data from the persisted index

In [10]:
question="Quels sont les critères pour la reconnaissance d'un élément de propriété, d'équipement et de matériel (PPE) comme actif selon les politiques comptables IFRS décrites ?"
query_engine = loaded_index.as_query_engine()
response = query_engine.query(question)
print(response)

Les critères pour la reconnaissance d'un élément de propriété, d'équipement et de matériel (PPE) comme actif selon les politiques comptables IFRS décrites incluent le fait que l'élément doit générer des avantages économiques futurs probables et que son coût peut être mesuré de manière fiable.


In [11]:
response.metadata

{'b77805d9-c6f6-45d0-acdd-e2fb409f6090': {'page_label': '19',
  'file_name': 'FY 2023 Communiqué de Presse.pdf',
  'file_path': '/home/cuphead/Projects/llama-index/notebooks/../data/FY 2023 Communiqué de Presse.pdf',
  'file_type': 'application/pdf',
  'file_size': 479112,
  'creation_date': '2024-08-07',
  'last_modified_date': '2024-02-15'},
 '1614a10d-f270-4a95-887f-844fe93e92c6': {'page_label': '19',
  'file_name': 'Orange-FY 2023 Communiqué de Presse.pdf',
  'file_path': '/home/cuphead/Projects/llama-index/notebooks/../data/Orange-FY 2023 Communiqué de Presse.pdf',
  'file_type': 'application/pdf',
  'file_size': 479112,
  'creation_date': '2024-08-07',
  'last_modified_date': '2024-02-15'}}

asking in french

In [12]:
question2="Quand Maroc Telecom a-t-il complètement intégré Gabon Telecom, et quelle acquisition significative Gabon Telecom a-t-il réalisée en 2016 ?"
query_engine = loaded_index.as_query_engine()
response = query_engine.query(question2)
print(response)

Maroc Telecom a complètement intégré Gabon Telecom en 2007. En 2016, Gabon Telecom a réalisé une acquisition significative en intégrant l'opérateur de téléphonie mobile Moov Gabon.


#### adding new documents

In [13]:
#new_documents=SimpleDirectoryReader("../delta_data").load_data()

In [14]:
#for doc in new_documents:
    #index.insert(doc)

asking about new documents

1. **Quelle a été l'évolution des revenus du Groupe Maroc Telecom en 2019 par rapport à 2018 ?**
   
   En 2019, le Groupe Maroc Telecom a généré des revenus totaux de 36 517 millions de MAD, ce qui représente une augmentation de 1,3% par rapport à 2018 (+0,9% à périmètre comparable). Cette performance est due à la croissance continue des activités au Maroc et à la résilience des activités internationales malgré la concurrence accrue et la pression réglementaire.

2. **Quel a été l'impact de la gestion des coûts sur l'EBITDA du Groupe Maroc Telecom en 2019 ?**
   
   Grâce à une gestion robuste des coûts, l'EBITDA du Groupe Maroc Telecom a atteint 18 922 millions de MAD en 2019, en hausse de 3,4% à périmètre comparable. La marge EBITDA a atteint 51,8%, en augmentation de 1,2 point à périmètre comparable.

3. **Quel est le montant des investissements réalisés par le Groupe Maroc Telecom en 2019 et comment se compare-t-il aux revenus ?**
   
   En 2019, les investissements en capital ont atteint 6 788 millions de MAD, ce qui représente une augmentation de 2,2% par rapport à l'année précédente. Ces investissements représentent 14,7% des revenus (hors fréquences et licences), ce qui est en ligne avec l'objectif déclaré pour l'année.

In [15]:
question3 = "Quelle a été l'évolution des revenus du Groupe Maroc Telecom en 2019 par rapport à 2018 ?"
query_engine = index.as_query_engine()
response = query_engine.query(question3)
print(response)

Les revenus du Groupe Maroc Telecom ont augmenté de 6,6% en 2019 par rapport à 2018.


In [16]:
response.metadata

{'b5e37790-050d-4280-9b2a-49aeeb1b4fea': {'page_label': '3',
  'file_name': 'FY 2023 Communiqué de Presse.pdf',
  'file_path': '/home/cuphead/Projects/llama-index/notebooks/../data/FY 2023 Communiqué de Presse.pdf',
  'file_type': 'application/pdf',
  'file_size': 479112,
  'creation_date': '2024-08-07',
  'last_modified_date': '2024-02-15'},
 '9d86fc74-caea-49c2-aba3-a4dc6b492140': {'page_label': '10',
  'file_name': 'FY 2023 Communiqué de Presse.pdf',
  'file_path': '/home/cuphead/Projects/llama-index/notebooks/../data/FY 2023 Communiqué de Presse.pdf',
  'file_type': 'application/pdf',
  'file_size': 479112,
  'creation_date': '2024-08-07',
  'last_modified_date': '2024-02-15'}}